In [2]:
!pip install keras-bert

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
#ÖNCEDEN EĞİTTİĞİMİZ MODELİ YÜKLÜYORUZ
!unzip /content/drive/My\ Drive/LANGUAGEmodel/700.000.zip

In [7]:
import numpy as np
from keras_bert import load_vocabulary, load_trained_model_from_checkpoint, Tokenizer, get_checkpoint_paths
import random

Using TensorFlow backend.


In [0]:
#EĞİTİLMİŞ BERT'İN PATH'İNİ BURADA BELİRTİYORUZ
pretrained_path = '/content/700.000'

In [12]:
paths = get_checkpoint_paths(pretrained_path)
paths

CheckpointPaths(config='/content/700.000/bert_config.json', checkpoint='/content/700.000/bert_model.ckpt', vocab='/content/700.000/vocab.txt')

In [13]:
model = load_trained_model_from_checkpoint(paths.config, paths.checkpoint, training=True, seq_len=None)
model.summary(line_length=120)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Model: "model_1"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               (None, None)               0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             (None, None)               0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, None, 768), (40000 30720000      Input-Token[0][0]                       
__________

In [0]:
#vocablary yüklüyoruz ver vocabdaki her kelime için bir index numarası atanıyor,sözlük oluşturuyoruz
token_dict = load_vocabulary(paths.vocab)
token_dict_inv = {v: k for k, v in token_dict.items()}
tokenizer = Tokenizer(token_dict)

In [0]:
import max_5_indeks as max5


In [0]:
#bu fonksiyonda cümle ve cümlede de maskelenip tahmin edilmesi istenen kelime verildiğinde, o kelime için tahmini gösterir
def predict_masked_top5(text,target):
  dogru = ''
  tokens = tokenizer.tokenize(text)
  target_tokens = tokenizer.tokenize(target)
  target_tokens.remove('[CLS]')
  target_tokens.remove('[SEP]')
  print('TARGET:',target_tokens)
  top1_score = 0
  top5_score = 0
  #eğer tahmin edilmesini istediğimiz kelime tek heceden oluşuyor ise
  if len(target_tokens) == 1:
    #maskelenecek olan kelimenin tokenlar arasında yerini buluyoruz 
    masked_indeks = tokens.index(target_tokens[0])
    #makelenmesini istediğimiz kelimenin bulunduğu yere [mask] yazıyoruz
    tokens[masked_indeks] = '[MASK]'
    #tokenların yerini sözlükten buluyoruz
    indices = np.array([[token_dict[token] for token in tokens]])
    #segment idsleri her token için 0 olarak yazıyotuz(tek cümlelelik denemeler için ), aynı cümlede olduğunu belirtir
    segments = np.array([[0] * len(tokens)])
    #maskelenen kelimenin yerleri bir olarak işaretlenir
    masks = np.array([[0] * len(tokens)])
    masks[0][masked_indeks]=1
    #tahmin değerleri alınıyor
    predicts = model.predict([indices, segments, masks])[0][0][masked_indeks].tolist()
    top5_tahminler = max5.max_n_elemet_indeks(predicts)
    tahmin = list(map(lambda x: token_dict_inv[x], top5_tahminler))
    for i in range(len(tahmin)):
      if tahmin[0] == target_tokens[0]:
        top1_score = 1
      if tahmin[i] == target_tokens[0]:
        dogru = tahmin[i]
        top5_score = 1
    
  
  #eğer tahmin edilecek kelime birden çok heceli ise
  else:
    masks = np.array([[0] * len(tokens)])
    for i in range(len(target_tokens)):
      masked_index = tokens.index(target_tokens[i])
      tokens[masked_index] = '[MASK]'
      masks[0][masked_index]=1
  
    indices = np.array([[token_dict[token] for token in tokens]])
    segments = np.array([[0] * len(tokens)])
    #maskelenmiş kelimenin tokenları için döndürülen tahminlerden maksimum beş tanesi tahmin değeri olarak alınıyor
    tahminler = []
    for i in range(masked_index-len(target_tokens)+1,masked_index+1):
      predicts = model.predict([indices, segments, masks])[0][0][i].tolist()
      tahminler.append(max5.max_n_elemet_indeks(predicts)) 
    #elde edilen tahmin indislerini kullanarak sözlükten hangi kelime olduğunu buluyoruz, token sayısı kadar dizi vardır
    # örnek: yemeden -> ['yeme', '##den'] -> [[1625, 9555, 4049, 2547, 7], [2548, 80, 1248, 364, 4582]](birinci token tahminler,ikinci token tahminler) 
    # -> [['##inde', 'gorursun', '##abilirsin', 'buldular', 'mi'], ['reklam', 'iki', 'rachel', 'gormek', 'yaniliyorsun']] 
    token_predicts = []
    for i in range(len(target_tokens)):
      token_predicts.append(list(map(lambda x: token_dict_inv[x], tahminler[i])))
    #tahminleri yazdırıyoruz
    tahmin = []
    for i in range(5):
      top5 = ''
      for j in range(len(target_tokens)):
        top5 += token_predicts[j][i] + ' '
      tahmin.append(top5)
      if top5 == target:
        dogru = top5
        top5_score = 1
        if i == 0:
          top1_score = 1
  print('TAHMINLER:',tahmin)  
  print('TOP1_SCORE:',top1_score)
  print('TOP5_SCORE:',top5_score)
  return top1_score,top5_score,tahmin,dogru


In [0]:
f = open("cleaned_test.txt","r")
test_samples = []
for line in f:
    test_samples .append(line)

In [17]:
len(test_samples)

199998

##**1- CUMLE UZUNLUKLARINA GORE MASKELENMIS KELİMELERİN TAHMİNİ BAŞARILARINI KARŞILAŞTIRMA**

In [14]:
# en uzun kelime uzunluğunu bulalım
lenght=[]
mx=len(test_samples[0])
index=0
for i in range(len(test_samples)):
      if mx<len(test_samples[i]):
         mx=len(test_samples[i])
         index=i
          
print("veri setindeki en uzun cümle:")
print(test_samples[index].split())
print("padding için kullanılacak maksimum kelime sayısı:",len(test_samples[index].split()))
print("index:",index)

veri setindeki en uzun cümle:
['yiyecekleri', 'sayayim', 'misir', 'arpa', 'yulaf', 'sarimsak', 'sogan', 'hardal', 'badem', 'fasulye', 'bezelye', 'barbunya', 'maydanoz', 'enginar', 'kivircik', 'salata', 'cay', 'beziryagi', 'biber', 'kestane', 'som', 'baligi', 'sardunya', 'ton', 'baligi', 'sut', 'albumini', 've', 'kazein', 'gravyer', 'peyniri', 'parmesan', 'peyniri', 'provolone', 'peyniri', 'hollanda', 'peyniri', 'italyan', 'damarli', 'peyniri', 've', 'domuz']
padding için kullanılacak maksimum kelime sayısı: 42
index: 117190


In [0]:
uzun = [] # cumledeki kelime sayısı 15 ve uzeri olanlar
orta = [] # cumledeki kelime sayısı 7'den buyuk ve 15'den kucuk olanlar
kısa = [] # cumledeki kelime sayısı 7'den kucuk esit ve 3'den buyuk esit olanlar
for i in range(len(test_samples)):
  text = test_samples[i]
  num_words = text.split()
  if (len(num_words) >= 15):
    uzun.append(text)
  elif(len(num_words) < 15 and len(num_words) > 7):
    orta.append(text)
  elif(len(num_words) <= 7 and len(num_words) >= 3):
    kısa.append(text)
  
  

In [16]:
print("UZUN CUMLE ORNEKK SAYISI: ",len(uzun))
print("ORTA UZUNLUK ORNEK SAYISI: ", len(orta))
print("KISA UZUNLUK ORNEK SAYISI: ", len(kısa))

UZUN CUMLE ORNEKK SAYISI:  1456
ORTA UZUNLUK ORNEK SAYISI:  19156
KISA UZUNLUK ORNEK SAYISI:  110185


In [17]:
orta = orta[:len(uzun)]
kısa = kısa[:len(uzun)]
print("UZUN CUMLE ORNEKK SAYISI: ",len(uzun))
print("ORTA UZUNLUK ORNEK SAYISI: ", len(orta))
print("KISA UZUNLUK ORNEK SAYISI: ", len(kısa))

UZUN CUMLE ORNEKK SAYISI:  1456
ORTA UZUNLUK ORNEK SAYISI:  1456
KISA UZUNLUK ORNEK SAYISI:  1456


In [0]:
def print_scores(tests,index_pos):
  score1 = 0
  score2 = 0
  
  for i in range(len(tests)):
    text = tests[i]
    words = text.split()
    if index_pos == 'b':
      index = 0
    elif index_pos == 'o':
      index = int(len(tests[i].split())/2)
    elif index_pos == 's':
      index = len(tests[i].split()) -1 
    else:
      index = random.randint(0,len(words)-1)
    target = words[index]
    print("===>DENEME:",i+1)
    print("TEXT:",text)
    top1, top5, tahminler,dogru = predict_masked_top5(text,target)
    score1 += top1
    score2 += top5
  print(len(tests),' ORNEKTEN', score1, 'TANESI DOGRU TOP1\'DE', score2, 'TANESI TOP5\'DE TAHMIN EDILMISTIR...')

In [0]:
#UZUN CUMLELER İCİN SONUCLAR
print_scores(uzun,'d')

In [0]:
#ORTA UZUNLUKTA CUMLE UZUNLUGUNDA TAHMİN SONUCLARI
print_scores(orta,'d')

In [0]:
# KISA UZUNLUKTA CUMLELER İCİN TAHMİN SCORELARI
print_scores(kısa,'d')

## 2- BAŞ, ORTA, SON MASKELENMİŞ KELİMELERİN TAHMİNLARİNİN KARŞILAŞTIRILMASI

In [0]:
# CÜMLELERDEKİ BAŞ -> 0 , ORTA -> int(LEN(cumle).split/2), SON -> len(cumle[i].split)


### A- UZUN CUMLELER ICIN BAS, ORTA, SON TAHMİN SONUCLARI

In [0]:
#UZUN CUMLELERDEKİ BAS KELİMELERİN TAHMİN SCORELARI
print_scores(uzun,'b')


In [0]:
#UZUN CUMLELER ICIN ORTA KELİMENIN TAHMİN SONUCLARI
print_scores(uzun,'o')

In [0]:
#UZUN CUMLE SON KELİMENİN TAHMİNLERİ
print_scores(uzun,'s')

### B- ORTA UZUNLUK CUMLELER BAS, ORTA, SON KELİMELERİN TAHMİNLERİ

In [0]:
#ORTA UZUNLUK CUMLELER BAS KELİME TAHMİNLERİ
print_scores(orta,'b')

In [0]:
#ORTA UZUNLUK ORTA KELİME TAHMİNİ
print_scores(orta,'o')

In [0]:
#ORTA UZUNLUK SON KELİME TAHMİNİ
print_scores(orta,'s')

### C- KISA UZUNLUK CUMLELERİ BAS, ORTA, SON KELİMLERİN TAHMİNLERİ

In [0]:
#KISA UZUNLUK BAS KELİME TAHMİNİ
print_scores(kısa,'b')

In [0]:
#KISA UZUNLUK ORTA KELİME TAHMİNİ
print_scores(kısa,'o')

In [0]:
#KISA UZUNLUK SON KELİME TAHMİNİ
print_scores(kısa,'s')

##3- KELİMELERİN TAHMİN EDİLME FREKANSLARI:

In [0]:
score1 = 0
score2 = 0
k=0
dogru_tahminler = []
for i in range(3000):
  test_sample_index = random.randint(0,len(test_samples)-1)
  text = test_samples[test_sample_index]
  sample_text_words = text.split()
  #test cümlesinin uzunluğu 3'ten fazla mı?
  if len(sample_text_words) >= 3:
    try:
      target_index = random.randint(0,len(sample_text_words))
      target = sample_text_words[target_index]
    except:
      k+=1
      continue
    print("===>DENEME:",i-k)
    print('TEXT:',text)
    top1_score, top5_score,tahminler, dogru = predict_masked_top5(text,target)
    score1 += top1_score
    score2 += top5_score
    if top5_score == 1:
      dogru_tahminler.append(dogru)
  else:
    k += 1
    continue
print(len(test_samples)-k,' ÖRNEKTEN ',score1,'TANESİ DOGRU TOP1\'DE',score2 ,' TANESİ TOP5\'DE TAHMİN EDİLMİŞTİR...')

===>DENEME: 0
TEXT: gel kucagima otur ki hissedebileyim

TARGET: ['kucagim', '##a']
TAHMINLER: ['ve bakalim ', 'nerede cenazen ', '##yim sallas ', '##dan neden ', 'uykular ediyorsunuz ']
TOP1_SCORE: 0
TOP5_SCORE: 0
===>DENEME: 1
TEXT: bes dakikaya geliriz

TARGET: ['dakika', '##ya']
TAHMINLER: ['buraya burada ', 'evlene kimildama ', 'kimse sur ', 'tecavuz tehlike ', 'bahce taniyan ']
TOP1_SCORE: 0
TOP5_SCORE: 0
===>DENEME: 2
TEXT: ama her zaman ise yarar

TARGET: ['yarar']
TAHMINLER: ['yarayacak', 'yaramaz', 'dinlemek', 'mahkemey', 'kron']
TOP1_SCORE: 0
TOP5_SCORE: 0
===>DENEME: 3
TEXT: bu bana eskileri animsatti

TARGET: ['bu']
TAHMINLER: ['simdi', 'otur', 'chantilas', 'onarim', 'ceza']
TOP1_SCORE: 0
TOP5_SCORE: 0
===>DENEME: 4
TEXT: artik kazi yapmazsaniz ellie yle ne yapacaksiniz

TARGET: ['yapacaksiniz']
TAHMINLER: ['giyecek', 'ogle', 'basima', 'uyanin', 'istersen']
TOP1_SCORE: 0
TOP5_SCORE: 0
===>DENEME: 5
TEXT: ama birisi onumde taytla geriniyorsa bir saati buluyor

TARGET: ['tay

In [0]:
from collections import Counter 
 
Counter = Counter(dogru_tahminler) 
  

most_occur = Counter.most_common(len(dogru_tahminler)) 
  
print(most_occur) 